In [1]:
import pandas as pd
import numpy as np
import sklearn
import time
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import KFold, train_test_split

In [2]:
t0 = time.time()
dataframe = pd.read_csv('aggregate-20160502.csv')
t1 = time.time()
print(str((t1-t0)/60) + " minutes")

5.535760736465454 minutes


In [3]:
df = dataframe.dropna(axis=0, how='all')

In [4]:
df['num_edits_binary'] = df['num_edits'].apply(lambda x: int(x > 0))
feature_names = df.columns[2:-1]
feature_names = feature_names.tolist()
label_name = 'num_edits_binary'

In [5]:
np.random.seed(seed=13579)
set1_idx = np.random.choice(range(len(df)), int(len(df) * .3), replace=False) #natural df to use for final test
set2_idx = list(set(range(len(df))) - set(list(set1_idx))) #for balanced set creating

In [ ]:
set2 = df.loc[set2_idx, list(df.columns.tolist())]
edited = set2[set2.num_edits > 0.0].copy(deep=True)
not_edited = set2[set2.num_edits == 0.0].copy(deep=True)
not_edited_selected = not_edited[0:len(edited)]
balanced_set = pd.concat([edited, not_edited_selected])

In [ ]:
balanced_set.to_csv("balanced_set_20160502.csv", sep=',')

NameError: name 'df' is not defined

In [ ]:
corr_table = df[feature_names+label_name].corr(method='pearson')
corr_table.style.background_gradient(cmap='RdYlGn', axis=1)

In [116]:
#from sklearn.utils import shuffle
#df = shuffle(df)

In [117]:
# X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df['num_edits_binary'],
#                                                     train_size=0.75, test_size=0.25)
# model = LogisticRegression()
# model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [118]:
# model.score(X_test, y_test)

0.97904687652275602

In [119]:
# y_pred = model.predict(X_test)
# y_pred.shape

(10261,)

In [120]:
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# tn, fp, fn, tp 

(10031, 17, 198, 15)

In [121]:
#z_pred = np.array([0]*len(y_pred))

In [46]:
# tn, fp, fn, tp = confusion_matrix(y_test, z_pred).ravel()
# tn, fp, fn, tp 

(10016, 0, 245, 0)

In [129]:
#model.classes_

array([0, 1])

In [130]:
# non_edited = []
# edited = []

In [146]:
# non_edited = model.predict_proba(X_test)[:, [0]]
# non_edited

array([[ 0.96302418],
       [ 0.82664788],
       [ 0.84415801],
       ..., 
       [ 0.95901964],
       [ 0.97770083],
       [ 0.86680648]])

In [148]:
# edited = model.predict_proba(X_test)[:, [1]]
# edited

array([[ 0.03697582],
       [ 0.17335212],
       [ 0.15584199],
       ..., 
       [ 0.04098036],
       [ 0.02229917],
       [ 0.13319352]])

In [159]:
# ROC_AUC_positive_class = metrics.roc_auc_score(y_test, edited)
# ROC_AUC_negative_class = metrics.roc_auc_score(y_test, non_edited)

In [161]:
# print("ROC AUC for positive class", ROC_AUC_positive_class)
# print("ROC AUC for negative class", ROC_AUC_negative_class)

ROC AUC for positive class 0.435123613229
ROC AUC for negative class 0.564878722975


Based on our ROC AUC scores, we can tell that our logistic regression model, trained on all the features and truncated data set (which contains a huge bias), predicts edited articles poorly, and non-edited - a little better. 

In [47]:
# FR and GB classifiers

In [86]:
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [87]:
#rf = RandomForestClassifier(bootstrap=True, max_depth=5)

In [88]:
#gb = GradientBoostingClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_depth=3)

In [89]:
# model_rf = rf.fit(X_train, y_train)
# y_pred_fr = model_rf.predict(X_test)

In [90]:
#rf_accuracy = accuracy_score(y_test, y_pred_fr)

In [91]:
# model_gb = gb.fit(X_train, y_train)
# y_pred_gb = model_gb.predict(X_test)

In [92]:
#gb_accuracy = accuracy_score(y_test, y_pred_gb)

In [93]:
# print('Random Forest accuracy: ' + str(rf_accuracy))
# print('Gradient Boosting accuracy: ' + str(gb_accuracy))

Random Forest accuracy: 1.0
Gradient Boosting accuracy: 1.0


The accuracy is too good to be true. Let's check if this will hold on a bigger data set and a smaller number of variables. Basically, let's take all the rows from our original data set and exlide the columns with NaNs. We'll be left with 24 variables.

In [111]:
#initial number of columns including the label one 
#len(dataframe.columns)

39

Let's create a label column now (binary) and a set of feature names 

In [109]:
# dataframe['num_edits_binary'] = dataframe['num_edits'].apply(lambda x: int(x > 0))
# new_feature_names = [x for x in dataframe.columns if 'size' not in x]
# new_feature_names = new_feature_names[2:-1]
# len(new_feature_names)

24

Since we're not deleting any rows and the indexes remain in order, we can run CV on our data. We'll run CV on a subset of train set, leaving dev set and final test set aside.

In [113]:
# from sklearn.model_selection import cross_val_score
# X_train, X_test, y_train, y_test = train_test_split(dataframe[new_feature_names], dataframe['num_edits_binary'], 
#                                                     test_size=0.2, random_state=0)

In [ ]:
# new_model = rf()
# >>> scores = cross_val_score(clf, iris.data, iris.target, cv=5)
# >>> scores   

In [ ]:
# KFold(n_splits=2, random_state=None, shuffle=False)

In [22]:
# linear regression

In [23]:
# k-nearest neighbors